# Import dependencies

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests

# SQLAlchemy
from sqlalchemy import create_engine
from secret import username, password
import psycopg2 

# Store CSV into DataFrame

In [2]:
accident_csv_file = "Resources/US_Accidents_June20.csv"
accident_data_df = pd.read_csv(accident_csv_file)
accident_data_df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [3]:
state_csv_file = "Resources/state_csvData.csv"
state_df = pd.read_csv(state_csv_file)
state_df.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [4]:
# Rename columns to match accident df for future merging
state_df = state_df.drop(columns={"Abbrev"})
state_df = state_df.rename(columns={"State": "state_name",
                                   "Code": "State"})
state_df.head()

,state_name,State
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


# accident_df

In [5]:
# view data before transformation
# accident_data_df.nunique()

In [6]:
# accident_data_df.info()

# Create new accident df

In [7]:
new_accident_data_df = accident_data_df[['ID', 'Start_Time', 'City', 'County', 'State', 'Zipcode', 'Street', 'Timezone', 'Temperature(F)', 'Visibility(mi)', 'Weather_Condition']].copy()
new_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy


# merge state and accident df

In [8]:
# Merge accident_df with state_df to obtain state_name for future merging with population df
clean_accident_data_df = pd.merge(new_accident_data_df, state_df, on= "State")
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast,Ohio
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy,Ohio
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy,Ohio


In [9]:
# convert start_time from object to datetime 
clean_accident_data_df['Start_Time'] = pd.to_datetime(clean_accident_data_df['Start_Time'])
clean_accident_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3513617 entries, 0 to 3513616
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ID                 object        
 1   Start_Time         datetime64[ns]
 2   City               object        
 3   County             object        
 4   State              object        
 5   Zipcode            object        
 6   Street             object        
 7   Timezone           object        
 8   Temperature(F)     float64       
 9   Visibility(mi)     float64       
 10  Weather_Condition  object        
 11  state_name         object        
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 348.5+ MB


In [10]:
# clean_accident_data_df.nunique()

In [11]:
# Limit data from 2016-2020 to 2016 only
clean_accident_data_df = clean_accident_data_df[(clean_accident_data_df['Start_Time']>= "2016-01-01") & 
                                            (clean_accident_data_df['Start_Time']<= "2018-12-31")]
# clean_accident_data_df.nunique()

In [12]:
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast,Ohio
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy,Ohio
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy,Ohio


# Check to see if all 2016 data is present

In [13]:
# Sort by datetime
clean_accident_data_df = clean_accident_data_df.sort_values('Start_Time')
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain,Ohio
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,45203,I-75 S,US/Eastern,36.0,10.0,Overcast,Ohio


In [14]:
# clean_accident_data_df.count()

In [15]:
# clean_accident_data_df.nunique()

# Test to see if correct data is included

In [16]:
# clean_accident_data_df['normalised_date'] = clean_accident_data_df['Start_Time'].dt.normalize()
# clean_accident_data_df.head()

# Reorganize df to look pretty

In [17]:
clean_accident_data_df = clean_accident_data_df[['ID', 'Start_Time', 'City', 'County', 'State', 'state_name', 'Zipcode', 'Street', 'Timezone', 'Temperature(F)', 'Visibility(mi)', 'Weather_Condition']]
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,state_name,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,Ohio,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,Ohio,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,Ohio,45203,I-75 S,US/Eastern,36.0,10.0,Overcast


In [18]:
clean_accident_data_df = clean_accident_data_df.rename(columns={"State": "state_abbrev",
                                                               "state_name":"State",
                                                               "Start_Time": "Date_Time"})


In [19]:
#import US state 2 letter abbreviation dictionary to create a state table 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [20]:
state_series = pd.Series(us_state_abbrev)

In [21]:
state_df = state_series.to_frame()

In [22]:
state_df.reset_index(inplace=True)

In [23]:
state_df.rename(columns={"index":"state",0:"statecode"}, inplace=True)

In [25]:
# state1_df = pd.merge(state_df,state_pop1_df, on="state" )

## Extract,Clean,Transform Accident data 

In [26]:
clean_accident_data_df.head()

,ID,Date_Time,City,County,state_abbrev,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,Ohio,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,Ohio,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,Ohio,45203,I-75 S,US/Eastern,36.0,10.0,Overcast


In [27]:
clean_accident_data_df_rename = clean_accident_data_df.copy()
clean_accident_data_df_rename = clean_accident_data_df_rename.rename(columns= {"ID": "id",
                                                                              "Date_Time": "datetime",
                                                                              "City": "city",
                                                                              "County": "county",
                                                                              "State": "state_name",
                                                                              "Zipcode": "zipcode",
                                                                              "Timezone": "timezone",
                                                                              "Temperature(F)": "temperature",
                                                                               "Visibility(mi)": "visibility",
                                                                               "Weather_Condition": "weather_condition"})
# drop street and state abbrev


In [28]:
clean_accident_data_df_rename.drop(columns= ["state_abbrev", "Street"], inplace=True)

In [29]:
clean_accident_data_df_rename.head()

,id,datetime,city,county,state_name,zipcode,timezone,temperature,visibility,weather_condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,Ohio,43017,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,Ohio,45424,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,Ohio,45424,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,Ohio,43068-3402,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,Ohio,45203,US/Eastern,36.0,10.0,Overcast


In [30]:
clean_accident_data_df_rename = clean_accident_data_df_rename.sort_values(by='datetime')
clean_accident_data_df_rename.head()

,id,datetime,city,county,state_name,zipcode,timezone,temperature,visibility,weather_condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,Ohio,43017,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,Ohio,45424,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,Ohio,45424,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,Ohio,43068-3402,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,Ohio,45203,US/Eastern,36.0,10.0,Overcast


In [31]:
clean_accident_data_df_rename = clean_accident_data_df_rename[['id', 'datetime', 'state_name', 'city', 'county', 'zipcode', 'timezone', 'temperature', 'visibility', 'weather_condition']]
clean_accident_data_df_rename.head()

,id,datetime,state_name,city,county,zipcode,timezone,temperature,visibility,weather_condition
45629,A-2478859,2016-02-08 00:37:08,Ohio,Dublin,Franklin,43017,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Ohio,Dayton,Montgomery,45424,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Ohio,Dayton,Montgomery,45424,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Ohio,Reynoldsburg,Franklin,43068-3402,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Ohio,Cincinnati,Hamilton,45203,US/Eastern,36.0,10.0,Overcast


In [34]:
# clean_accident_data_df_rename.nunique()

## Run Accident_Schema_Final.sql in Postgres now

In [35]:
# clean_accident_data_df_rename.to_csv("Resources/Output/accident.csv", index=False)

In [36]:
rds_connection_string =f'{username}:{password}@localhost:5432/us_accidents'

In [37]:
engine = create_engine(f'postgresql+psycopg2://{rds_connection_string}')

In [38]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  database "us_accidents" does not exist

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
clean_accident_data_df_rename.info()

In [ ]:
clean_accident_data_df_rename.to_sql(name='accident', con=engine, if_exists='append', index=False)